In [1]:
# REMPLIR LES MÉTADONNEES avant de tout executer !

NAME_CORPUS = "DATACATLOGUE_NER"
VERSION = 1.0

# créer le dossier :
# - conlls : contient tous les conll

In [2]:
# Installer les librairies
!pip install pandas
!pip install numpy
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from collections import Counter
from datetime import datetime
import json

import pandas as pd, numpy as np
from nltk.tree import Tree
from nltk.chunk import conlltags2tree

In [4]:
basepath = "/content"
path_to_conlls = f"{basepath}/conlls"

In [5]:
# merger tous les conll en un seul "all.conll"
!cat $path_to_conlls/*.conll >> $basepath/all.conll

In [6]:
with open(f'{basepath}/all.conll', "r", encoding="utf-8") as f:
    conll = f.read()

print(conll)

CATALOGUE O
D'UNE O
COLLECTION O
DE O
MONNAIES O
FRANÇAISES O
ROYALES O
& O
SEIGNEURIALES O
ET O
DE O
MEDAILLES O
ROMAINES O
DONT O
LA O
VENTE O
AUX O
ENCHÈRES O
PUBLIQUES O
AURA O
LIEU O
HOTEL B-LOC
DES I-LOC
COMMISSAIRES I-LOC
- I-LOC
PRISEURS I-LOC
Rue B-LOC
Drouot I-LOC
, I-LOC
n I-LOC
° I-LOC
5 I-LOC
SALLE O
N O
" O
6 O
, O
AU O
1er O
Le O
Samedi O
11 O
Février O
1860 O
Par O
le O
ministère O
de O
Me O
DELBERGUE-CORMONT B-PER
, O
Cre-Priseur O
, O
rue B-LOC
de I-LOC
Provence I-LOC
, I-LOC
8 I-LOC
. O

Assisté O
de O
M O
. O

ROLLIN B-PER
, O
Expert O
, O
rue B-LOC
Vivienne I-LOC
, I-LOC
12 I-LOC
. O

RENOU B-ORG
& I-ORG
MAULDE I-ORG
IMPRIMEURS O
DE O
LA O
COMPAGNIE O
DES O
COMMISSAIRES-PRISEURS O
rue B-LOC
de I-LOC
Rivoli I-LOC
, I-LOC
144 I-LOC
1860 O
CONDITIONS O
DE O
LA O
VENTE O
Elle O
sera O
faite O
au O
comptant O
. O

Les O
acquéreurs O
paieront O
, O
en O
sus O
des O
adjudications O
, O
cinq O
pour O
cent O
applicables O
aux O
frais O
de O
vente O
. O

CATALOGUE O
DE O
MON

In [7]:
sentences = 1
sentences_annotated = 1
tokens = []
tags = []
tokens_tagged = []
# Liste intermediare pour tester si la phrases contient des mentions annotées.
inter_sentences_annotated = []

for row in conll.splitlines():
    # si méthode n°1 : sep = "\t"
    # si méthode n°2 : sep = " "
    pair = row.split(" ")
    # Si utilisation de la méthode n°2 :
    # Si la longueur de la liste est égale à 1, cela signifie 
    # que l'on passe à une nouvelle phrase. 
    if len(pair) == 1:
    # Si utilisation de la méthode n°1 : 
    #if pair[0] == "" and pair[1] == "":
        sentences += 1
        if len(inter_sentences_annotated) > 0:
            sentences_annotated += 1
            inter_sentences_annotated = []
        else:
            inter_sentences_annotated = []
    else:
        mention = pair[0]
        ner_tag = pair[1]
        tokens.append(mention)
        tags.append(ner_tag)
        if ner_tag != "O":
            inter_sentences_annotated.append(ner_tag)
            mention_tagged = mention
            tokens_tagged.append(mention_tagged)

In [8]:
print(len(tokens_tagged))
print(tokens[0:20])
print(tags[0:20])

# sanity check
assert len(tokens) == len(tags), 'length between tokens and tags is not the same.'

1462
['CATALOGUE', "D'UNE", 'COLLECTION', 'DE', 'MONNAIES', 'FRANÇAISES', 'ROYALES', '&', 'SEIGNEURIALES', 'ET', 'DE', 'MEDAILLES', 'ROMAINES', 'DONT', 'LA', 'VENTE', 'AUX', 'ENCHÈRES', 'PUBLIQUES', 'AURA']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [9]:
# "_" correpond au POS Tags. Par défaut, le CONLL Reader de NLTK accepte trois valeur (Mention, POS, NER Tags)
conlltags = [(token, "_", tg) for token, tg in zip(tokens, tags)]
ne_tree = conlltags2tree(conlltags)
original_text = []
for subtree in ne_tree:
    # On passe les tags "O"
    if type(subtree) == Tree:
        original_label = subtree.label()
        original_string = " ".join([token for token, _ in subtree.leaves()])
        original_text.append((original_string, original_label))


iob = dict(Counter(tags))
merge_iob = dict(Counter([t[1] for t in original_text]))

del iob['O']
iob

{'B-LOC': 467,
 'B-LOCderiv': 1,
 'B-ORG': 28,
 'B-PER': 574,
 'B-null': 7,
 'I-LOC': 80,
 'I-ORG': 25,
 'I-PER': 278,
 'I-null': 2}

In [10]:
out = {
    "name corpus": NAME_CORPUS,
    "version": VERSION,
    "date":datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
    "statistics":{
     "Total sentences":sentences,
     "Total sentences annotated":sentences_annotated,
     "Total tokens annotated (IOB)":sum(iob.values()),
     "Total tokens annotated (merge IOB)":sum(merge_iob.values()),
     "Total tokens annotated details (IOB)": iob,
     "Total tokens annotated details (merge IOB)": merge_iob  
    }
}

out

{'date': '07/07/2022 12:20:09',
 'name corpus': 'DATACATLOGUE_NER',
 'statistics': {'Total sentences': 3122,
  'Total sentences annotated': 880,
  'Total tokens annotated (IOB)': 1462,
  'Total tokens annotated (merge IOB)': 1079,
  'Total tokens annotated details (IOB)': {'B-LOC': 467,
   'B-LOCderiv': 1,
   'B-ORG': 28,
   'B-PER': 574,
   'B-null': 7,
   'I-LOC': 80,
   'I-ORG': 25,
   'I-PER': 278,
   'I-null': 2},
  'Total tokens annotated details (merge IOB)': {'LOC': 467,
   'LOCderiv': 1,
   'ORG': 28,
   'PER': 576,
   'null': 7}},
 'version': 1.0}

In [11]:
# Write recipe outputs
with open(f'{basepath}/metrics.json', mode="w") as f:
    json.dump(out, f)